<a href="https://colab.research.google.com/github/solsa24/AI-Chat-Assistant/blob/master/AI_Agent_with_Gemini_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import google.generativeai as genai
import json
import time

# --- Part 0: Configure the LLM ---
try:
    # Get the API key from Codespaces secrets
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        raise ValueError("GEMINI_API_KEY not found. Please set it in your Codespaces secrets.")
    genai.configure(api_key=api_key)
    print("Gemini API configured successfully.")
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    exit()


# --- Part 1: MCP (Modular Code Package) Builder ---

class MCPBuilder:
    """
    Builds, reuses, and manages Modular Code Packages (MCPs).
    """
    def __init__(self):
        self._registry = {}

    def create_mcp(self, name: str, description: str, backend_tech: str, frontend_tech: str):
        """Creates a new MCP and adds it to the registry."""
        if name in self._registry:
            print(f"MCP '{name}' already exists. Reusing.")
            return self._registry[name]

        mcp = {
            "name": name,
            "description": description,
            "backend": {"tech": backend_tech, "files": []},
            "frontend": {"tech": frontend_tech, "files": []},
        }
        self._registry[name] = mcp
        print(f"MCP '{name}' created.")
        return mcp

    def get_mcp(self, name: str):
        """Retrieves an MCP from the registry."""
        return self._registry.get(name)

# --- Part 2: Code Generator ---

class CodeGenerator:
    """
    Generates code by calling the Gemini LLM.
    """
    def __init__(self):
        # Initialize the Gemini model
        self.model = genai.GenerativeModel('gemini-1.5-pro-latest')
        print("Gemini 1.5 Pro model initialized.")

    def _llm_generate_code(self, prompt: str) -> str:
        """
        Calls the Gemini API to generate code.
        Includes error handling and retries.
        """
        print(f"  - Sending prompt to Gemini: '{prompt[:50]}...'")
        try:
            response = self.model.generate_content(prompt)
            # Add a small delay to respect API rate limits
            time.sleep(2)
            return response.text
        except Exception as e:
            print(f"  - An error occurred with the Gemini API: {e}")
            return f"# Error generating code: {e}"

    def generate_backend(self, mcp: dict):
        """Generates backend code for a given MCP."""
        print(f"\nGenerating {mcp['backend']['tech']} backend for {mcp['name']}...")

        main_py_prompt = f"Generate a complete, runnable main.py file for a FastAPI application for '{mcp['description']}'. Include basic CRUD endpoints for recipes. The code should be fully functional."
        models_py_prompt = f"Generate a models.py file for a FastAPI application for '{mcp['description']}'. It should include a Pydantic model for a 'Recipe' with fields: name (str), ingredients (list[str]), and instructions (str)."

        main_py_code = self._llm_generate_code(main_py_prompt)
        models_py_code = self._llm_generate_code(models_py_prompt)

        mcp["backend"]["files"] = [
            {"name": "main.py", "content": main_py_code},
            {"name": "models.py", "content": models_py_code},
        ]
        print(f"Backend for {mcp['name']} generated.")
        return mcp

    def generate_frontend(self, mcp: dict):
        """Generates frontend code for a given MCP."""
        print(f"\nGenerating {mcp['frontend']['tech']} frontend for {mcp['name']}...")

        index_js_prompt = f"Generate a complete, runnable pages/index.js file for a Next.js frontend for '{mcp['description']}'. It should be a simple landing page that displays a title and a brief welcome message. Use functional components and basic JSX."

        index_js_code = self._llm_generate_code(index_js_prompt)

        mcp["frontend"]["files"] = [
            {"name": "pages/index.js", "content": index_js_code}
        ]
        print(f"Frontend for {mcp['name']} generated.")
        return mcp

# --- Part 3: Orchestrator ---

class AIAgent:
    """
    The main AI Agent that orchestrates the app creation process.
    """
    def __init__(self):
        self.mcp_builder = MCPBuilder()
        self.code_generator = CodeGenerator()

    def create_app(self, description: str, backend_tech: str, frontend_tech: str):
        """
        The one-command function to generate the full-stack application.
        """
        print("--- Starting AI Agent for Full-Stack App Development ---")

        app_mcp = self.mcp_builder.create_mcp(
            name="RecipeAppCore",
            description=description,
            backend_tech=backend_tech,
            frontend_tech=frontend_tech,
        )

        self.code_generator.generate_backend(app_mcp)
        self.code_generator.generate_frontend(app_mcp)

        self._save_code(app_mcp)

        print("\n--- ✅ Full-Stack App Generation Complete! ---")
        print("\nGenerated file structure:")
        print("""
recipe_app/
├── backend/
│   ├── main.py
│   └── models.py
└── frontend/
    └── pages/
        └── index.js
""")
        print("\nTo explore the files, check the file explorer on the left.")
        print("To test the backend, you would typically run: 'cd recipe_app/backend && pip install fastapi uvicorn && uvicorn main:app --reload'")

    def _save_code(self, mcp: dict):
        """Saves the generated code to the filesystem."""
        base_dir = "recipe_app"
        # Clean up old directory if it exists
        if os.path.exists(base_dir):
            import shutil
            shutil.rmtree(base_dir)

        backend_dir = os.path.join(base_dir, "backend")
        frontend_dir = os.path.join(base_dir, "frontend", "pages")

        os.makedirs(backend_dir, exist_ok=True)
        os.makedirs(frontend_dir, exist_ok=True)

        for file_info in mcp["backend"]["files"]:
            # Clean up markdown code blocks if the LLM includes them
            content = file_info["content"].replace("```python", "").replace("```", "").strip()
            with open(os.path.join(backend_dir, file_info["name"]), "w") as f:
                f.write(content)

        for file_info in mcp["frontend"]["files"]:
            content = file_info["content"].replace("```javascript", "").replace("```jsx", "").replace("```", "").strip()
            with open(os.path.join(frontend_dir, os.path.basename(file_info["name"])), "w") as f:
                f.write(content)

if __name__ == "__main__":
    agent = AIAgent()
    agent.create_app(
        description="A recipe web app with user login, recipe search by ingredients, social sharing, and ratings",
        backend_tech="FastAPI",
        frontend_tech="Next.js",
    )